# Lesson 6

In this tutorial we elaborate a way to establish a lookup table for a person name string variable, which consists of multiple components, i.e. first name, middle name, last name.

As a library we use _metaphone_ which may not be installed in your environment. When you are based on Anaconda python envrionment you may have to install the package via the _pip_ command as described here

* https://www.puzzlr.org/install-packages-pip-conda-environment/

`/Users/felix/anaconda3/bin/pip install metaphone`
 

## Introduction

Suppose that the person is called _Peter Alfred Escher_ and we got that name via an official API.

We use the standard library _itertools_ to calcutlate all  combinations and we apply then the doublemetaphone algorithm.

In [1]:
import itertools
from metaphone import doublemetaphone

Suppose we retrieved the full qualified name in one of our variables and we have also a unique identifier of this person.

In [2]:
person_qualified_name = "Peter Alfed Escher"

As a fresh up,  we use the _doubelmetaphone_ alogrithm to generate a tuple of keys which can be used for the comparison. Refer to the tutorial __[Medium Towards Data Science Tutorial](https://towardsdatascience.com/python-tutorial-fuzzy-name-matching-algorithms-7a6f43322cc5)__

## Should we use permutations or combinations

What should we use for our name component algorithm, `permutations` or `combinations` ?

Let's quickly check the differences by applying the relevant functions in the `itertools` library.

* Assume we have the name "peter alfred escher".

Let's apply first the `combinations` function:

In [22]:
name_list = list( itertools.combinations(["peter", "alfred", "escher"],2))
print(name_list)

[('peter', 'alfred'), ('peter', 'escher'), ('alfred', 'escher')]


As well as the `permutations` function:

In [4]:
name_list = list( itertools.permutations(["peter", "alfred", "escher"],2))
print(name_list)

[('peter', 'alfred'), ('peter', 'escher'), ('alfred', 'peter'), ('alfred', 'escher'), ('escher', 'peter'), ('escher', 'alfred')]


As you can see within the permutation the **order of a name component plays a role**. There is a tuple for ('peter','alfred') as well as one for ('alfred','peter'). Within the combination the **order of a name component is irrelevant**.

For us the order will not play a role, 'peter escher' is treated as 'escher peter'. We will sort the name components before we apply the double methaphone algorithm.


## Build up a Lookup Directory 
We now to build up a lookup directory. Each name tuple combination we concatenate into a single string, generate a metaphone tuple and add it to our directory. As value we store our unique person identifier. 

When we generate our name string we ensure that the elements are sorted.

In [5]:
def generate_normalized_name(name_tuple):
    name_arr = list(name_tuple)
    name_arr.sort()
    name_str = ''.join(name_arr)
    return name_str.lower() 

In [6]:
generate_normalized_name(("Peter","Alfred","Escher"))

'alfredescherpeter'

In [7]:
def add_combinations_to_directory(name_tuple_list,person_id):
    for name_tuple in name_tuple_list:
        name_str = generate_normalized_name(name_tuple)
        metaphone_tuple = doublemetaphone(name_str)
        lookup_dict[0][metaphone_tuple[0]] = [person_id]
        lookup_dict[1][metaphone_tuple[1]] = [person_id]

In [8]:
lookup_dict = ({},{})

name_list = list( itertools.combinations(["peter", "alfred", "escher"],2))
print("Name combinations: "+str(name_list))
add_combinations_to_directory(name_list, 'A123')
print("Lookup directory: "+lookup_dict.__str__())

Name combinations: [('peter', 'alfred'), ('peter', 'escher'), ('alfred', 'escher')]
Lookup directory: ({'ALFRTPTR': ['A123'], 'AXRPTR': ['A123'], 'ALFRTXR': ['A123']}, {'': ['A123'], 'ASKRPTR': ['A123'], 'ALFRTSKR': ['A123']})


As one can see in the above output: The value of the dictionary is a list element `[person_id]` . We are doing that because potentially we want to store a second name combination which will map to the same key. Therefore we have to ensure that we can store multiple person identifiers in the value. More about that later.

The above name list is not complete yet. What we said is that we want to compare any combination. At the moment we have only added combinations with two elements.
Our complete name list would look as follows (3 elements):

In [9]:
name_list = [('peter','alfred','escher')]
name_list.extend(itertools.combinations(["peter", "alfred", 'escher'],2))
name_list.extend(itertools.combinations(["peter", "alfred", 'escher'],1))
print(name_list)

[('peter', 'alfred', 'escher'), ('peter', 'alfred'), ('peter', 'escher'), ('alfred', 'escher'), ('peter',), ('alfred',), ('escher',)]


Let's add our tuple list to our lookup directoy

In [10]:
lookup_dict = ({},{})
add_combinations_to_directory(name_list, 'A123')
print("Lookup directory: "+lookup_dict.__str__())

Lookup directory: ({'ALFRTXRPTR': ['A123'], 'ALFRTPTR': ['A123'], 'AXRPTR': ['A123'], 'ALFRTXR': ['A123'], 'PTR': ['A123'], 'ALFRT': ['A123'], 'AXR': ['A123']}, {'ALFRTSKRPTR': ['A123'], '': ['A123'], 'ASKRPTR': ['A123'], 'ALFRTSKR': ['A123'], 'ASKR': ['A123']})


Let's enhance our function `add_combinations_to_directory` in order to handle the existence of dictionary keys and values:
* check if the key already exists
* in case the key already exists, check if the value array already contains `person_id`
* in not, then add the `person_id`to the value arry

In [11]:
def add_combinations_to_directory(name_tuple_list,person_id):
    for name_tuple in name_tuple_list:
        concat_name = generate_normalized_name(name_tuple)
        metaphone_tuple = doublemetaphone(concat_name)
        if metaphone_tuple[0] in lookup_dict[0]:
            if not person_id in lookup_dict[0][metaphone_tuple[0]]:
                lookup_dict[0][metaphone_tuple[0]].append(person_id)
        else:
            lookup_dict[0][metaphone_tuple[0]] = [person_id]
        if metaphone_tuple[1] in lookup_dict[1]:
            if not person_id in lookup_dict[1][metaphone_tuple[1]]:
                lookup_dict[1][metaphone_tuple[1]].append(person_id)
        else:
            lookup_dict[1][metaphone_tuple[1]] = [person_id]

In [12]:
name_list = list( itertools.combinations(["peter", "alfred", 'king'],2))
print(name_list)
add_combinations_to_directory(name_list, 'A555')
print("Lookup directory: "+lookup_dict.__str__())

[('peter', 'alfred'), ('peter', 'king'), ('alfred', 'king')]
Lookup directory: ({'ALFRTXRPTR': ['A123'], 'ALFRTPTR': ['A123', 'A555'], 'AXRPTR': ['A123'], 'ALFRTXR': ['A123'], 'PTR': ['A123'], 'ALFRT': ['A123'], 'AXR': ['A123'], 'KNKPTR': ['A555'], 'ALFRTKNK': ['A555']}, {'ALFRTSKRPTR': ['A123'], '': ['A123', 'A555'], 'ASKRPTR': ['A123'], 'ALFRTSKR': ['A123'], 'ASKR': ['A123']})


As one can see in the above example. Certain keys are now pointing to two person identifiers, e.g. `'ALFRTPTR': ['A123', 'A555']`

## Generalize our Name Component Permutation Generator

Up to now we were preparing our `name_list` manually for 3,2 and 1 element(s). Let's get that implemented into a function as well. We define a general `generate_combinations` function, which calculates all permutations for `n` name components passed in via the `name_list`:

In [13]:
def generate_combinations(name_tuple):
    perms = []
    perms.append(name_tuple)
    i = len(list(name_tuple))-1
    while i > 0:
        perms.extend(itertools.combinations(name_tuple,i))
        i -=1
    return perms

In [14]:
perms = generate_combinations(("peter","alfred","escher"))
perms.__str__()

"[('peter', 'alfred', 'escher'), ('peter', 'alfred'), ('peter', 'escher'), ('alfred', 'escher'), ('peter',), ('alfred',), ('escher',)]"

Let's define the overall function which will call the two functions we defined:

In [15]:
def add_person_to_lookup_directory(person_id, name_tuple):
    combs = generate_combinations(name_tuple) 
    add_combinations_to_directory(combs,person_id)

In [16]:
lookup_dict = ({},{})
add_person_to_lookup_directory("A123",("peter", "alfred", "escher"))
add_person_to_lookup_directory("A235",("peter", "escher"))
add_person_to_lookup_directory("A345",("peter", "john", "good"))
print("Lookup directory: "+lookup_dict.__str__())

Lookup directory: ({'ALFRTXRPTR': ['A123'], 'ALFRTPTR': ['A123'], 'AXRPTR': ['A123', 'A235'], 'ALFRTXR': ['A123'], 'PTR': ['A123', 'A235', 'A345'], 'ALFRT': ['A123'], 'AXR': ['A123', 'A235'], 'KTJNPTR': ['A345'], 'JNPTR': ['A345'], 'KTPTR': ['A345'], 'KTJN': ['A345'], 'JN': ['A345'], 'KT': ['A345']}, {'ALFRTSKRPTR': ['A123'], '': ['A123', 'A235', 'A345'], 'ASKRPTR': ['A123', 'A235'], 'ALFRTSKR': ['A123'], 'ASKR': ['A123', 'A235'], 'ANPTR': ['A345'], 'AN': ['A345']})


As an example: "peter" as a single component has as key the value `PTR` and as value list the three identifiers: ` 'PTR': ['A123', 'A235', 'A345']`

So we have  a function ready which populates our lookup table. Now we have to design a `matchName` function which will check if the name or its combination of it  matching to any lookup table entry

## Creating a "Matching Name" Function
In this first iteration we construct a `match_list` which will store all lookup entries which are matching any permutations of our `name_list` 

In [17]:
def match_name(name_tuple):
    match_list = []
    combinations = generate_combinations(name_tuple) 
    for comb_tuple in combinations:
        concat_name = generate_normalized_name(comb_tuple)
        metaphone_tuple = doublemetaphone(concat_name)
        if metaphone_tuple[0] in lookup_dict[0]:
            match_list.append((concat_name, lookup_dict[0][metaphone_tuple[0]]))
    print("Match with "+ match_list.__str__())

In [18]:
match_name(["peter","alfred"])

Match with [('alfredpeter', ['A123']), ('peter', ['A123', 'A235', 'A345']), ('alfred', ['A123'])]


As we can see, we have two tuples which point to one `person_id` and one tuple `peter` which points to 3 persons (Obviously surnames will be re-used by multiple persons). The two tuples pointing to one person have  the **same** id `'A123'`. That means our match identified exactly one person. Would we have single person tuples in our result which are pointing to different persons would mean our match isn't unique. 

So we enhance our method to do this uniquness check, as well:

* Do we have in our match list one or multiple tuples which always point to one single person?
* If yes we a found a unique record otherwise we return _None_


In [19]:
def match_name(name_tuple):
    match_list = []
    combinations = generate_combinations(name_tuple) 
    for comb_tuple in combinations:
        concat_name = generate_normalized_name(comb_tuple)
        metaphone_tuple = doublemetaphone(concat_name)
        if metaphone_tuple[0] in lookup_dict[0]:
            match_list.append((concat_name, lookup_dict[0][metaphone_tuple[0]]))
    # Iterate through all matches and check for single result tuples
    # Ensure that the singe result tuples are pointing to the same id
    # If not or no single result tuple exists, return 'None'
    unique_id = None
    for match_tuple in match_list:
        if len(match_tuple[1]) == 1:
            if  unique_id is not None:
                if unique_id != match_tuple[1][0]:
                    unique_id = None
                    break
            else:
                unique_id = match_tuple[1][0]
    return unique_id, match_list
                     


In [20]:
id = match_name(("peter","alfred"))
print("Unique Id (expect A123): "+id[0])
id = match_name(("peter","john"))
print("Unique Id (expect A345): "+id[0])
id = match_name(("peter","escher"))
print("Unique Id (expect None): "+str(id[0]))

Unique Id (expect A123): A123
Unique Id (expect A345): A345
Unique Id (expect None): None


## Enhancing our Function with the Ranking Feature

As explained in our last tutorial, the ranking capability of our alogrithm is rather limited, nevertheless we want to used its feature as well in our `match_name`function.

In [ ]:
from enum import Enum

class Threshold(Enum):
    WEAK = 0
    NORMAL = 1
    STRONG = 2
    
def double_metaphone_compare(tuple1,tuple2,threshold):
    if threshold == Threshold.STRONG:
        if tuple1[0] == tuple2[0]:   
            return (True,Threshold.STRONG)
    elif threshold == Threshold.NORMAL:
        if tuple1[0] == tuple2[1] or tuple1[1] == tuple2[0]:
            return (True,Threshold.NORMAL)
    else:
        if tuple1[1] == tuple2[1]:
            return (True,Threshold.WEAK)
    return False